In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_base_path = "/content/drive/MyDrive/data-process"

mineru_zip_path = f"{drive_base_path}/MinerU.zip"

print("Unzipping MinerU from Google Drive...")
!unzip -q -o "{mineru_zip_path}" -d /content/

%cd /content/MinerU

print("\nInstalling dependencies...")
!pip install -e .
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install doclayout_yolo ultralytics transformers ftfy pyclipper PyMuPDF

print("\n✅ Setup Complete!")

Unzipping MinerU from Google Drive...
/content/MinerU

Installing dependencies...
Obtaining file:///content/MinerU
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.6/786.6 kB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 136.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os, doclayout_yolo, fileinput
package_path = os.path.dirname(doclayout_yolo.__file__)
tasks_file_path = os.path.join(package_path, 'nn', 'tasks.py')
original_line = 'ckpt = torch.load(file, map_location="cpu")'
corrected_line = 'ckpt = torch.load(file, map_location="cpu", weights_only=False)'
with fileinput.FileInput(tasks_file_path, inplace=True) as file:
    for line in file:
        print(line.replace(original_line, corrected_line), end='')
print("✅ Successfully patched tasks.py.")

✅ Successfully patched tasks.py.


In [ ]:


import os
import shutil
import pandas as pd
import fitz



drive_base_path = "/content/drive/MyDrive/data-process"

input_pdf_path = os.path.join(drive_base_path, "Chemistry2e-WEB.pdf")
csv_path = os.path.join(drive_base_path, "chapter_ranges_chemistry.csv")
subject_name = "chemistry"

page_output_dir = os.path.join(drive_base_path, f"output/{subject_name}_pages")
chapter_output_dir = os.path.join(drive_base_path, f"output/{subject_name}_chapters")
temp_base_dir = "/content/temp_processing"



os.makedirs(page_output_dir, exist_ok=True)
os.makedirs(chapter_output_dir, exist_ok=True)
os.makedirs(temp_base_dir, exist_ok=True)
print(f"✅ Setup complete.")
print(f"Page-by-page output will be saved to: {page_output_dir}")
print(f"Final chapter output will be saved to: {chapter_output_dir}\n")


print("--- STAGE 1: Starting Page-by-Page Extraction ---")

try:
    with fitz.open(input_pdf_path) as doc:
        total_pages = len(doc)
    print(f"PDF '{os.path.basename(input_pdf_path)}' has {total_pages} pages.")
except Exception as e:
    print(f"Error opening PDF: {e}")
    total_pages = 0

for page_index in range(total_pages):
    page_number_for_filename = page_index + 1
    page_filename = f"page_{str(page_number_for_filename).zfill(4)}.md"
    destination_file = os.path.join(page_output_dir, page_filename)

    if os.path.exists(destination_file):
        print(f"⏩ Skipping Page {page_number_for_filename} (already exists in Drive).")
        continue

    temp_output_path = os.path.join(temp_base_dir, f"temp_page_{page_number_for_filename}")
    print(f"--- Processing Page {page_number_for_filename}/{total_pages} ---")

    command = (
        f"mineru -p '{input_pdf_path}' "
        f"-o '{temp_output_path}' "
        f"-l en "
        f"-s {page_index} -e {page_index}"
    )
    !{command}

    try:
        pdf_basename = os.path.splitext(os.path.basename(input_pdf_path))[0]
        source_file = os.path.join(temp_output_path, pdf_basename, "auto", f"{pdf_basename}.md")
        shutil.move(source_file, destination_file)
        shutil.rmtree(temp_output_path)
        print(f"✅ Successfully created and saved to Drive: {destination_file}\n")
    except FileNotFoundError:
        print(f"⚠️ Warning: Could not find output file for page {page_number_for_filename}.")

print("--- ✅ STAGE 1 Complete! All pages extracted to Google Drive. ---\n")

# --- STAGE 2: AGGREGATE PAGES INTO CHAPTERS ---
print("--- STAGE 2: Starting Chapter Aggregation ---")
chapters_df = pd.read_csv(csv_path)

print("--- Aggregating: chapter_00_introduction ---")
first_chapter_start_page = chapters_df.iloc[0]['start']
intro_chapter_filepath = os.path.join(chapter_output_dir, "chapter_00_introduction.md")
with open(intro_chapter_filepath, 'w', encoding='utf-8') as intro_file:
    for page_num in range(1, first_chapter_start_page):
        page_filename = f"page_{str(page_num).zfill(4)}.md"
        page_filepath = os.path.join(page_output_dir, page_filename)
        try:
            with open(page_filepath, 'r', encoding='utf-8') as page_file:
                intro_file.write(page_file.read())
            intro_file.write(f"\n\n--- End of Page {page_num} ---\n\n")
        except FileNotFoundError:
            print(f"  - Warning: Page file not found for introduction: {page_filepath}")
print(f"✅ Successfully created and saved to Drive: {intro_chapter_filepath}\n")

for index, row in chapters_df.iterrows():
    chapter_title = row['title']
    start_page = row['start']
    end_page = row['end']
    clean_chapter_name = f"chapter_{index + 1:02d}.md"
    chapter_filepath = os.path.join(chapter_output_dir, clean_chapter_name)
    print(f"--- Aggregating: {clean_chapter_name}: {chapter_title} ---")
    with open(chapter_filepath, 'w', encoding='utf-8') as chapter_file:
        for page_num in range(start_page, end_page + 1):
            page_filename = f"page_{str(page_num).zfill(4)}.md"
            page_filepath = os.path.join(page_output_dir, page_filename)
            try:
                with open(page_filepath, 'r', encoding='utf-8') as page_file:
                    chapter_file.write(page_file.read())
                chapter_file.write(f"\n\n--- End of Page {page_num} ---\n\n")
            except FileNotFoundError:
                print(f"  - Warning: Page file not found: {page_filepath}")
    print(f"✅ Successfully created and saved to Drive: {chapter_filepath}\n")

print("--- 🎉 All chapters processed and saved to Google Drive! ---")

Streaming output truncated to the last 5000 lines.
Fetching 1 files: 100% 1/1 [00:00<00:00, 20661.60it/s]
Table-ocr rec en: 100% 11/11 [00:00<00:00, 109.39it/s]
Table-wireless Predict: 100% 2/2 [00:00<00:00, 24.78it/s]
Table-wired Predict:   0% 0/2 [00:00<?, ?it/s]
Fetching 1 files: 100% 1/1 [00:00<00:00, 21183.35it/s]
Table-wired Predict: 100% 2/2 [00:00<00:00,  4.23it/s]
OCR-det Predict:   0% 0/1 [00:00<?, ?it/s]
Fetching 1 files: 100% 1/1 [00:00<00:00, 19784.45it/s]

Fetching 1 files: 100% 1/1 [00:00<00:00, 17924.38it/s]
OCR-det Predict: 100% 1/1 [00:02<00:00,  2.00s/it]
Processing pages:   0% 0/1 [00:00<?, ?it/s]
Fetching 2 files: 100% 2/2 [00:00<00:00, 35394.97it/s]
Processing pages: 100% 1/1 [00:01<00:00,  1.05s/it]
OCR-rec Predict: 100% 2/2 [00:00<00:00, 42.59it/s]
2025-10-14 14:17:26.288 | INFO     | mineru.cli.common:_process_output:155 - local output dir is /content/temp_processing/temp_page_1111/Chemistry2e-WEB/auto
✅ Successfully created and saved to Drive: /content/drive/M